In [ ]:
import os 
import time

import ants
from antspynet.utilities import deep_atropos, preprocess_brain_image

In [2]:
raw_t1 = ants.image_read("t1.nii.gz")


Preprocessing should be consist with the training data consisted of:
* n4 bias correction,
* denoising,
* brain extraction
* affine registration to MNI.

``` python
# Preprocessing, no need to do this, just preprocess the image in deep_atropos

preprocessed_image = preprocess_brain_image(raw_t1,verbose=False, do_bias_correction=True, do_denoising=True,
                        brain_extraction_modality='t1', template_transform_type='Affine')
preprocessed_image['preprocessed_image'].to_file('t1_brain.nii.gz')
preprocessed_image['brain_mask'].to_file('t1_brain_mask.nii.gz')
preprocessed_t1 = ants.image_read("t1_brain.nii.gz")
```

In [ ]:
atropos = deep_atropos(raw_t1,do_preprocessing=True, use_spatial_priors=1, antsxnet_cache_directory=None, verbose=False)

In [4]:
outfile='seg_mask.nii.gz'
atropos['segmentation_image'].to_file(outfile)

Perform Atropos-style six tissue segmentation using deep learning.

The labeling is as follows: 
* Label 0 : background 
* Label 1 : CSF 
* Label 2 : gray matter 
* Label 3 : white matter 
* Label 4 : deep gray matter 
* Label 5 : brain stem  
* Label 6 : cerebellum



In [5]:
label = 2 # gray matter

In [6]:
mask_file = f'mask_label{label}.nii.gz' 

In [7]:
! fslmaths {outfile} -thr {label} -uthr {label} -bin {mask_file}

In [8]:
! fslmaths t1.nii.gz -mas {mask_file} gray_matter.nii.gz